<a href="https://colab.research.google.com/github/yibowang15/Capstone/blob/VGG16-Initialization/Method1_Green_Channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow.keras.applications

**Import VGG 16 Models**

In [4]:
from tensorflow.keras.applications import VGG16

# Load VGG16 model, but exclude the classifer on the top layer
# Weight is pre-trained using imagenet dataset
# Classifier on top layer is excluded because customization(for binary classification) needed
# Specifiy input_shape and pooling because include_top is set to False
base_model = VGG16(weights='imagenet', include_top = False, input_shape= (224, 224, 3))

# Check: print the summary of the model to ensure the top layer has been successfully removed
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

**Modify VGG16 Model Input - Expected Input (224, 224, 1)**

In [3]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# Create a new input parameter, which is (224, 224, 1) to adapt to grayscale images
input_tensor = Input(shape = (224, 224, 1), name = 'Input Layer')

# Replace the original (224, 224, 3) input and output to (224, 224, 1)
modified_model = Model(inputs=input_tensor, outputs=base_model(input_tensor))

# Check: check the input shape
modified_model.summary()

**Modify the weight of the first layer - Take the Weights of the Green Channel**


In [11]:
import numpy as np

# To get the weights of the first convolutional kernel
conv1_weights = base_model.layers[1].get_weights()

# Check: check original kernel shape before adjustment
print("Original kernel shape:", conv1_weights[0].shape)

# Get the weights of the Green channel (assume Green index is 1)
green_channel_weights = conv1_weights[0][:, :, 1, :]

# Expand the dimension to satisfy the expectation of VGG16 for the filter
# (Given we actually removed the input_channel in the previous step)
green_channel_weights = np.expand_dims(green_channel_weights, axis=2)
new_kernel = green_channel_weights

# Check: check if new kernel dimensions become [3, 3, 1, 64] after being averaged
print("New kernel shape after adjustment:", new_kernel.shape)

modified_model.summary()

Original kernel shape: (3, 3, 3, 64)
New kernel shape after adjustment: (3, 3, 1, 64)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

**Add binary classifier on top of the model**

In [12]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Add Flatten Layer
x = Flatten(name = 'Flatten')(modified_model.output)  # Flatten

# Add two fully connected layers (4096 hidden units) and their corresponding Dropout Layers
x = Dense(4096, activation='relu', name = 'FC1')(x)
x = Dropout(0.5, name = 'Dropout1')(x)  # Dropout ratio set to 0.5
x = Dense(4096, activation='relu', name = 'FC2')(x)
x = Dropout(0.5, name = 'Dropout2')(x)

# Add binary classification dense layer(Using Sigmoid)
x = Dense(1, activation='sigmoid', name = 'Classification')(x)

# Add binary classification dense layer(Using Softmax)
# x = Dense(2, activation='softmax', name='Classification')(x)

# Create new model
final_model = Model(inputs=modified_model.input, outputs=x)

# Check: check the details of the final model
final_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout1 (Dropout)                   │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC2 (Dense)                          │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout2 (Dropout)                   │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Classification (Dense)               │ (None, 1)                   │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 134,264,641 (512.18 MB)

 Non-trainable params: 0 (0.00 B)

**Compile the final model: Initialize Learning Rate, Momentum, and Loss Function**

In [14]:
from tensorflow.keras.optimizers import SGD

# Initialize learning rate and momentum
opt = SGD(learning_rate = 0.01, momentum=0.9)

# Use loss = 'binary_crossentropy' (Sigmoid)
final_model.compile(optimizer= opt, loss='binary_crossentropy', metrics=['accuracy'])

# Use loss = 'categorical_crossentropy'（Softmax)
#final_model.complie(optimizer = opt, loss='categorical_crossentropy', metrics = ['accuracy'])

**Start the Training**

In [ ]:
# Assume train_data 和 validation_data are pre-processed dataset

# final_model.fit(train_data, epochs=10, validation_data=validation_data)